In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

In [2]:
# External dependencies
import os

import cudf  # cuDF is an implementation of Pandas-like Dataframe on GPU
import rmm

import numpy as np
import nvtabular as nvt

from sklearn.model_selection import train_test_split

In [3]:
INPUT_DATA_DIR = os.environ.get(
    "INPUT_DATA_DIR", os.path.expanduser("./data/")
)

In [4]:
rmm.reinitialize(managed_memory=True)

## Read Movie and Rating Features

In [5]:
movie_features = cudf.read_parquet(os.path.join(INPUT_DATA_DIR, "movie_features.parquet"))
movie_features = movie_features.drop(labels=["datetime", "created"], axis=1)
movie_features.head()

,tags_unique,movie_id,genres,tags_nunique
0,"[40345, 59079]",27265,[9],2
1,[28414],27273,[8],1
2,[32292],27266,[9],1
3,"[3365, 33048, 43053, 46467, 50807, 51310, 5739...",27282,"[9, 18]",10
4,"[31336, 34155, 42699, 48919, 48957, 51656, 602...",27290,[1],8


In [6]:
ratings_features = cudf.read_parquet(os.path.join(INPUT_DATA_DIR, "ratings_features.parquet"))
ratings_features.head()

,day,interaction,user_id,movie_id,rating,timestamp
0,4146,True,1,293,5.0,1147880044
1,4146,True,1,303,3.5,1147868817
2,4146,True,1,304,5.0,1147868828
3,4146,True,1,655,5.0,1147878820
4,4146,True,1,879,3.5,1147868510


## Join Ratings With Movie Features

In [7]:
joined_features = movie_features.merge(ratings_features)
joined_features.head()

,tags_unique,movie_id,genres,tags_nunique,day,interaction,user_id,rating,timestamp
0,"[171, 1136, 2817, 5546, 5707, 5711, 6224, 7375...",5242,"[2, 3, 17, 18]",83,5242,True,69,4.5,1242591289
1,"[2816, 6022, 6600, 28007, 33271, 39734, 42248,...",9544,"[6, 9, 16]",13,5243,True,69,5.0,1242647324
2,"[640, 783, 2061, 2170, 2175, 2817, 4427, 5693,...",6259,"[3, 4, 5, 6]",101,5242,True,69,3.5,1242591122
3,[6222],11157,[8],1,5243,True,69,5.0,1242647426
4,"[256, 1961, 2817, 4087, 5449, 6191, 6218, 6224...",5271,"[2, 3, 17, 13]",113,5242,True,69,4.0,1242591718


In [8]:
sampled_indices = np.array([np.random.randint(0,nunique) if nunique > 0 else -1 for nunique in joined_features["tags_nunique"].to_pandas()], dtype=np.int32)

In [9]:
sampled_tags = np.array([tags[index] if index >= 0 else 0 for index, tags in zip(sampled_indices, joined_features["tags_unique"].to_pandas())], dtype=np.int32)

In [10]:
sampled_tags.dtype

dtype('int32')

In [11]:
joined_features["search_term"] = sampled_tags

In [12]:
joined_features["genre"] = joined_features["genres"]

In [13]:
joined_features.head()

,tags_unique,movie_id,genres,tags_nunique,day,interaction,user_id,rating,timestamp,search_term,genre
0,"[171, 1136, 2817, 5546, 5707, 5711, 6224, 7375...",5242,"[2, 3, 17, 18]",83,5242,True,69,4.5,1242591289,18451,"[2, 3, 17, 18]"
1,"[2816, 6022, 6600, 28007, 33271, 39734, 42248,...",9544,"[6, 9, 16]",13,5243,True,69,5.0,1242647324,39734,"[6, 9, 16]"
2,"[640, 783, 2061, 2170, 2175, 2817, 4427, 5693,...",6259,"[3, 4, 5, 6]",101,5242,True,69,3.5,1242591122,41673,"[3, 4, 5, 6]"
3,[6222],11157,[8],1,5243,True,69,5.0,1242647426,6222,[8]
4,"[256, 1961, 2817, 4087, 5449, 6191, 6218, 6224...",5271,"[2, 3, 17, 13]",113,5242,True,69,4.0,1242591718,54241,"[2, 3, 17, 13]"


In [14]:
joined_features = joined_features.drop(labels=["tags_unique", "tags_nunique", "interaction", "rating", "genres"], axis=1)

In [15]:
joined_features = joined_features.sort_values("timestamp")

In [16]:
joined_features.head()

,movie_id,day,user_id,timestamp,search_term,genre
204186,47,0,2262,789652009,44917,"[15, 18]"
1262211,57,385,13424,822873600,31280,[9]
1262233,11,385,13424,822873600,62334,"[6, 9, 16]"
4139124,32,385,42937,822873600,62107,"[15, 17, 18]"
9906401,18,385,102689,822873600,2370,[6]


In [17]:
joined_features["search_term"].dtype

dtype('int32')

In [18]:
training_examples = joined_features.groupby(["user_id", "day"]).agg(
    {
        "search_term": "collect",
        "genre": "collect",
        "timestamp": "collect",
        "movie_id": ["collect", "count"]
    }
)

In [19]:
training_examples.head()

search_term  \
                                                        collect   
user_id day                                                       
1       4146  [17594, 40872, 33339, 57686, 24188, 53948, 397...   
2       4071  [54673, 19997, 44011, 67380, 36218, 48078, 469...   
3       7521  [38027, 927, 30849, 71320, 48032, 41101, 43529...   
        7688  [39905, 26066, 62981, 47185, 41868, 39232, 668...   
        8045  [591, 51913, 67271, 43501, 35740, 20286, 45708...   

                                                          genre  \
                                                        collect   
user_id day                                                       
1       4146  [[3, 10], [9, 17, 18], [3, 9, 19], [2, 3, 6, 1...   
2       4071  [[3, 10], [6, 16], [2, 3, 17, 18], [9, 17, 18]...   
3       7521  [[6, 9, 16, 19], [7, 12, 18], [3, 6, 17], [3, ...   
        7688  [[7, 9, 17, 18], [2, 9, 19], [6, 16], [8], [20...   
        8045  [[7, 15, 18], [2, 18], [6, 7, 18], [4, 17], [6...   

                                                      timestamp  \
                                                        collect   
user_id day                                                       
1       4146  [1147868053, 1147868097, 1147868414, 114786846...   
2       4071  [1141415528, 1141415566, 1141415576, 114141558...   
3       7521  [1439472199, 1439472203, 1439472211, 143947221...   
        7688  [1453904021, 1453904031, 1453904046, 145390404...   
        8045  [1484753654, 1484753762, 1484753766, 148475380...   

                                                       movie_id        
                                                        collect count  
user_id day                                                            
1       4146  [5841, 1592, 1218, 6417, 6259, 3353, 1062, 879...    53  
2       4071  [5841, 493, 1339, 1592, 2550, 5338, 150, 234, ...   125  
3       7521  [352, 586, 1238, 1, 2481, 258, 315, 1167, 523,...   315  
        7688  [1176, 1178, 10678, 9777, 11446, 11930, 10407,...    20  
        8045  [1063, 29365, 3908, 726, 763, 110, 213, 29375,...    48

In [20]:
len(training_examples[training_examples[("movie_id", "count")] > 1])

639668

In [21]:
training_examples = training_examples[training_examples[("movie_id", "count")] > 1]

In [22]:
training_examples["target_item"] = training_examples[("movie_id", "collect")].list.get(-1)

In [23]:
training_examples[("target_item",)] = training_examples["target_item"]
training_examples = training_examples.drop(labels="target_item", axis=1)

In [24]:
training_examples.columns = ["_".join(list(column_name)) for column_name in training_examples.columns]
training_examples.columns = [column_name.replace("_collect", "s") for column_name in training_examples.columns]

In [25]:
training_examples.head()

search_terms  \
user_id day                                                       
1       4146  [17594, 40872, 33339, 57686, 24188, 53948, 397...   
2       4071  [54673, 19997, 44011, 67380, 36218, 48078, 469...   
3       7521  [38027, 927, 30849, 71320, 48032, 41101, 43529...   
        7688  [39905, 26066, 62981, 47185, 41868, 39232, 668...   
        8045  [591, 51913, 67271, 43501, 35740, 20286, 45708...   

                                                         genres  \
user_id day                                                       
1       4146  [[3, 10], [9, 17, 18], [3, 9, 19], [2, 3, 6, 1...   
2       4071  [[3, 10], [6, 16], [2, 3, 17, 18], [9, 17, 18]...   
3       7521  [[6, 9, 16, 19], [7, 12, 18], [3, 6, 17], [3, ...   
        7688  [[7, 9, 17, 18], [2, 9, 19], [6, 16], [8], [20...   
        8045  [[7, 15, 18], [2, 18], [6, 7, 18], [4, 17], [6...   

                                                     timestamps  \
user_id day                                                       
1       4146  [1147868053, 1147868097, 1147868414, 114786846...   
2       4071  [1141415528, 1141415566, 1141415576, 114141558...   
3       7521  [1439472199, 1439472203, 1439472211, 143947221...   
        7688  [1453904021, 1453904031, 1453904046, 145390404...   
        8045  [1484753654, 1484753762, 1484753766, 148475380...   

                                                      movie_ids  \
user_id day                                                       
1       4146  [5841, 1592, 1218, 6417, 6259, 3353, 1062, 879...   
2       4071  [5841, 493, 1339, 1592, 2550, 5338, 150, 234, ...   
3       7521  [352, 586, 1238, 1, 2481, 258, 315, 1167, 523,...   
        7688  [1176, 1178, 10678, 9777, 11446, 11930, 10407,...   
        8045  [1063, 29365, 3908, 726, 763, 110, 213, 29375,...   

              movie_id_count  target_item  
user_id day                                
1       4146              53         7237  
2       4071             125         2061  
3       7521             315        24542  
        7688              20         4240  
        8045              48         9335

In [26]:
def remove_last_from_col(df, field_name):
    df[field_name] = np.array([values[:-1] for values in df[field_name].to_pandas()], dtype=object)
    return df

In [27]:
training_examples = remove_last_from_col(training_examples, "search_terms")
training_examples = remove_last_from_col(training_examples, "genres")
training_examples = remove_last_from_col(training_examples, "timestamps")
training_examples = remove_last_from_col(training_examples, "movie_ids")

In [28]:
training_examples["movie_id_count"] = training_examples["movie_id_count"] - 1

In [29]:
def flatten_genres(g):
    if len(g) > 0:
        return np.concatenate(g).ravel()
    else:
        return []

training_examples["genres"] = \
    training_examples["genres"].to_pandas().map(flatten_genres)

In [30]:
training_examples.head()

search_terms  \
user_id day                                                       
1       4146  [17594, 40872, 33339, 57686, 24188, 53948, 397...   
2       4071  [54673, 19997, 44011, 67380, 36218, 48078, 469...   
3       7521  [38027, 927, 30849, 71320, 48032, 41101, 43529...   
        7688  [39905, 26066, 62981, 47185, 41868, 39232, 668...   
        8045  [591, 51913, 67271, 43501, 35740, 20286, 45708...   

                                                         genres  \
user_id day                                                       
1       4146  [3, 10, 9, 17, 18, 3, 9, 19, 2, 3, 6, 10, 3, 4...   
2       4071  [3, 10, 6, 16, 2, 3, 17, 18, 9, 17, 18, 2, 3, ...   
3       7521  [6, 9, 16, 19, 7, 12, 18, 3, 6, 17, 3, 4, 5, 6...   
        7688  [7, 9, 17, 18, 2, 9, 19, 6, 16, 8, 20, 6, 10, ...   
        8045  [7, 15, 18, 2, 18, 6, 7, 18, 4, 17, 6, 7, 9, 7...   

                                                     timestamps  \
user_id day                                                       
1       4146  [1147868053, 1147868097, 1147868414, 114786846...   
2       4071  [1141415528, 1141415566, 1141415576, 114141558...   
3       7521  [1439472199, 1439472203, 1439472211, 143947221...   
        7688  [1453904021, 1453904031, 1453904046, 145390404...   
        8045  [1484753654, 1484753762, 1484753766, 148475380...   

                                                      movie_ids  \
user_id day                                                       
1       4146  [5841, 1592, 1218, 6417, 6259, 3353, 1062, 879...   
2       4071  [5841, 493, 1339, 1592, 2550, 5338, 150, 234, ...   
3       7521  [352, 586, 1238, 1, 2481, 258, 315, 1167, 523,...   
        7688  [1176, 1178, 10678, 9777, 11446, 11930, 10407,...   
        8045  [1063, 29365, 3908, 726, 763, 110, 213, 29375,...   

              movie_id_count  target_item  
user_id day                                
1       4146              52         7237  
2       4071             124         2061  
3       7521             314        24542  
        7688              19         4240  
        8045              47         9335

In [31]:
training_examples.dtypes

search_terms       list
genres             list
timestamps         list
movie_ids          list
movie_id_count    int32
target_item       int32
dtype: object

In [33]:
training_examples.to_parquet(os.path.join(INPUT_DATA_DIR, "training_examples.parquet"))

In [34]:
rmm.reinitialize(managed_memory=False)